In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.autograd import Variable
from torchvision import datasets, models, transforms
#from torcheval.metrics.functional import multiclass_f1_score
import os
import numpy as np
import random

In [2]:
import torch
print(torch.cuda.is_available())


False


/home/phil/miniconda3/envs/thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
data_path = '../Data/full dataset'
small_data_path = '../Data/Small Data 1000'
extra_small_data_path = '../Data/Small Dataset 200'
figures_output_path = '../Outputs/figures'
csv_outputs ='../Outputs/csv'
models_output_path = '../Models'
model_checkpoints_path = '../Models/checkpoints'


In [3]:
# Create transforms
data_transforms = {'train': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                   'val': transforms.Compose([
                      transforms.Resize((300, 300)),
                      transforms.ToTensor(),
                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                   'test': transforms.Compose([
                      transforms.Resize((300,300)),
                      transforms.ToTensor(),
                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}


In [4]:
# Create datasets


image_datasets = {x:datasets.ImageFolder(os.path.join(data_path, x), data_transforms[x]) for x in [ 'val', 'test']}
image_datasets['train'] = datasets.ImageFolder(os.path.join(extra_small_data_path,'train'), data_transforms['train'])



In [5]:
# Data loaders

dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True) for x in ['train', 'val', 'test']}


In [6]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

In [7]:
print(f"Classes in the dataset are:{class_names}")

print(f"Num batches in training dataset:{len(dataloaders['train'])}")
print(f"Num images in training dataset:{dataset_sizes['train']}")

print(f"Num batches in val dataset:{len(dataloaders['val'])}")
print(f"Num images in val dataset:{dataset_sizes['val']}")

print(f"Num batches in test dataset:{len(dataloaders['test'])}")
print(f"Num images in test dataset:{dataset_sizes['test']}")



Classes in the dataset are:['0', '1', '2', '3', '4']
Num batches in training dataset:63
Num images in training dataset:1000
Num batches in val dataset:363
Num images in val dataset:5796
Num batches in test dataset:363
Num images in test dataset:5796


Resnet 50 <br>
V2 weights are improved upon V1 version

In [8]:
#load model
#model_conv = torchvision.models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
model_conv = torchvision.models.inception_v3(weights='Inception_V3_Weights.IMAGENET1K_V1')
#model_conv = torchvision.models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1')
#model_conv = torchvision.models.efficientnet_v2_l(weights='EfficientNet_V2_L_Weights.IMAGENET1K_V1')

In [9]:
# Freeze layers in the model to prevent disturbing the weights
for param in model_conv.parameters():
    param.required_grad=False

In [10]:
# Replace final layer with new one with 5 output nodes
#n_inputs = model_conv.fc.in_features
#model_conv.fc= nn.Linear(in_features=n_inputs, out_features=len(class_names))
# For VGG16 it is movel_conv.classifier that is replaced
#model_conv.classifier[-1] = nn.Linear(in_features=4096, out_features=len(class_names))


In [11]:
if torch.cuda.is_available():
    model_conv.cuda()

/home/phil/miniconda3/envs/thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
# Set up loss function and optimiser
#model_conv = torch.load(os.path.join(models_output_path, 'ResNet50_2.pt'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_conv.fc.parameters(), lr=0.1, momentum=0.9)
#optimizer = optim.SGD(model_conv.classifier.parameters(), lr=0.01, momentum=0.9)
exp_lr_sch = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#f1 = F1Score(task="multiclass", num_classes=3)


In [13]:
num_epoch = 50
train_loss=[]
train_accuracy=[]
val_loss=[]
val_accuracy=[]

for epoch in range(num_epoch):
    exp_lr_sch.step()
    iterations=0
    iter_loss=0.0
    correct=0

    model_conv.train()

    for images, labels in dataloaders['train']:
        images = Variable(images)
        labels=Variable(labels)
        if torch.cuda.is_available():
            images=images.cuda()
            labels=labels.cuda()

        optimizer.zero_grad()
        outputs,_= model_conv(images)
        loss=criterion(outputs, labels)
        iter_loss+=loss.item()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs,1)
        correct+=(predicted==labels).sum()
        iterations+=1

    train_loss.append(iter_loss/iterations)
    train_iter_acc = 100*correct/dataset_sizes['train']
    train_accuracy.append(train_iter_acc)
    print(f"Epoch {epoch+1}/{num_epoch}, Loss {loss.item()}, train accuracy {train_iter_acc}")
    #torch.save(model_conv, os.path.join(models_output_path, 'Inception_200.pt'))
    """torch.save({
            'epoch': epoch,
            'model_state_dict': model_conv.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, PATH)
"""
    if 1==1:
        model_conv.eval()
        test_loss=0.0
        correct=0
        iterations=0

        for images, labels in dataloaders['val']:
            images = Variable(images)
            labels=Variable(labels)
            if torch.cuda.is_available():
                images=images.cuda()
                labels=labels.cuda()


            outputs= model_conv(images)
            loss=criterion(outputs, labels)
            iter_loss+=loss.item()
            _, predicted = torch.max(outputs,1)
            correct+=(predicted==labels).sum()
            iterations+=1

        val_acc = 100*correct/dataset_sizes['val']
        print(f"Epoch {epoch+1}/{num_epoch}, Loss {loss.item()}, val accuracy {val_acc}")
        val_loss.append(iter_loss/iterations)
        val_iter_acc = 100*correct/dataset_sizes['val']
        val_accuracy.append(val_iter_acc)
    else:
        pass



/home/phil/miniconda3/envs/thesis/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1/50, Loss 14.000083923339844, train accuracy 25.600000381469727


KeyboardInterrupt: 

In [ ]:
import pandas as pd

train_accuracy = [i.item() for i in train_accuracy]
val_accuracy = [i.item() for i in val_accuracy]
pd.DataFrame([train_loss, train_accuracy, val_loss, val_accuracy]).to_excel(os.path.join(figures_output_path, 'Inception_200_200_stats.xlsx'))
pd.DataFrame([predicted, labels]).to_excel(os.path.join(figures_output_path, 'Inception_200_200_stats.xlsx'))

In [ ]:
outputs

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
train_accuracy

In [ ]:
pd.DataFrame([train_loss, train_accuracy, val_loss, val_accuracy])

In [ ]:
model_conv(images).shape